<a href="https://colab.research.google.com/github/RayCyder/Intro/blob/main/notear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

线性因果结构探索，基于notear

# 环境配置

## 下载代码

In [1]:
!git clone https://github.com/xunzheng/notears.git --depth=1 || echo "Repository already cloned or another error occurred."

Cloning into 'notears'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 1), reused 12 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 19.81 KiB | 9.90 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
!ls notears/

bin  Dockerfile  experiments  LICENSE  notears	README.md  setup.py


## 安装依赖

In [4]:
!pip3 install  --no-cache  numpy \
                                      scipy \
                                      python-igraph \
                                      tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.2 MB/s eta 0:00:00


# 代码

暂时未用
$$
H2=−logdet(sI−W\circ W)+dlogs
$$

In [6]:
import numpy as np
import scipy.linalg as slin
import scipy.linalg as sla
import numpy.linalg as la
import math
def H2(B, s=1.0):
    d = B.shape[0]
    Id = np.eye(d)
    W = B*0.5
    #−logdet(sI−W◦W)+dlogs
    M = s * Id - W * W
    h = - la.slogdet(M)[1]
    if math.isinf(h):
        print(h)
    h = h+ d * np.log(s)
    if h < 0:
        print(h)
    #G_h = 2 * W * sla.inv(M).T
    return h#, G_h

$$
H=e^B-d,B=W>=tau
$$

In [7]:
def H(B):
    e_B = slin.expm(B*B) #
    return np.trace(e_B)-B.shape[0]
def gradient_H(B):
    pass

In [8]:
def B(W,tau=0.01):
    value = (np.abs(W)>=tau).astype(int)
    return value

梯度下降法求解F,暂时忽略1范数:
$$
F(W) = \ell(W; \mathbf{X}) + \lambda \| W \|_1 = \frac{1}{2n} \| \mathbf{X} - \mathbf{X} W \|_F^2 + \lambda \| W \|_1  \\
$$
$$
\nabla F(W)=-\frac{1}{n}X^{T}(X-XW)
$$



In [9]:

def gradient_F(X,W):
    gradient = -(1 / n) * X.T @ (X - X @ (W))
    return gradient
def F(X,W):
    return np.linalg.norm(X-X@W,ord='fro')**2

In [10]:
def solve_F(X,W,B=None,step=0.001,iter=100,tol=1e-6):
    if B is None:
        B = np.ones_like(W)
    d = X.shape[1]
    # W = np.zeros((d,d))
    for i in range(iter):
        g = gradient_F(X,W)
        W = W - step*g*B
        error = np.linalg.norm(step*g*B,ord='fro')
        if  error<  tol:
            break
    print(error,F(X,W))
    return W

## 主流程
* 设置W初始值
* 主循环
* 遍历W d*d个元素，根据W[i,j]与tau值的大小关系，翻转当前元素W[i,j]
 * 保证翻转后，罚函数H(B)的取值严格降低，否则会死循环
 * 计算F(W),并记录最小值对应的 i_min,j_min,W_min
* 更新W = W_min
* 如果罚函数H(B)=0 结束循环，并返回W

### 讨论：
* 考虑F(W)无H(W)约束下，直接梯度下降法，得到的F(W)是最小的，
在加入H(W)约束后，会导致F(W)增加，优选选择使得F(W)上升最慢的方向，同时保证H(W)严格下降
* 如果直接用阿格朗日法求解，存在F增长，与H下降两者难均衡问题，一个思路是不是增加\alpha \rho,而是减小；或者增加参数\beta 来控制F增加过快导致无法充分搜索W空间的问题；
* 之前有思路是将W矩阵空间看成是不同的DAG等价类组成的集合：称之为马尔可夫毯集合，遍历其中元素寻找最符合评分函数的那个类，再在类中按照某个方式选最合适的那个DAG。问题是：这种搜索完全回避了非DAG空间，导致无环约束H(W)的存在成为鸡肋；

In [18]:
def solve_h(W, X,tau=0.01,alpha=0.1,rho=1,max_iter=1000):
    #如果能够在所有的DAG里面进行搜索，就会好很多。
    # 如果所有的DAG都是极值点的话。。。。
    n, d = X.shape
    W = solve_F(X,W)#相当于带约束求解； 能否假设，带约束的解空间最小值大于不带约束的解空间的最小值；
    W_old = W.copy()
    #B = (np.abs(W)>=tau).astype(int)
    #B = np.ones((d,d))
    last_i = -1
    last_j = -1
    for it in range(max_iter):
        least_F = 1e10 #F(X,W)
        new_last_i = -1
        new_last_j = -1
        Best_W = W.copy()
        for i in range(d):
            for j in range(d):
                if i ==  last_i and j == last_j:
                    continue
                # search from unconstrained solution,try to satisfy the constraint
                Wp = W.copy()
                #flip W
                if W[i,j] >= tau or W[i,j] <= -tau:
                    W[i,j] = 0
                else:
                    W[i,j] = W_old[i,j]
                    if W[i,j] < tau and W[i,j] > -tau:
                        assert np.sign(W[i,j]) !=0
                        W[i,j] = tau*np.sign(W[i,j])

                    # W[j,i] = 0
                # assert F will increase and H will decrease
                # assert  zeros in unconstrained min value of W  is also zeoro in real W,
                # so H must decrease. generally init value of W is full dense graph
                if H(B(W)) >= H(B(Wp)):
                # it seems that always circle
                #,for example  W[0,1]=1, W[1,0]=1 need break it
                #,so maybe H should rigorly decrease
                    W = Wp
                    continue
                # find the value F increase least
                new_F = F(X,W) #+ alpha*H(B)+0.5*rho*H(B)**2
                print(i,j,W[i,j],new_F)
                if least_F > new_F:
                    least_F = new_F
                    Best_W = W
                    new_last_i = i
                    new_last_j = j
                W = Wp
        W = Best_W.copy()
        last_i = new_last_i
        last_j = new_last_j
        print('iter:',it,least_F,H(B(W)),"chosen:",last_i,last_j,W[last_i,last_j])
        print(W)
        if H(B(W)) == 0:
            break
    return W*B(W)

In [19]:
def solve_all(X,lamda=0.1):
    X = X - np.mean(X, axis=0, keepdims=True)
    d = X.shape[1]
    W = np.zeros((d,d))
    B = np.ones((d,d))
    #loss = np.norm(X-X@W,ord='fro')**2 #减均值，除方差；
    W = solve_h(W,X)
    print(W)
    #X-X@W=(I-W)X=E n个独立的高斯分布；计算每一行与高斯分布的相似性；均值方差；
    #独立性假设，与最小二乘法；
    return W

In [20]:
#main
import sys
sys.path.insert(0, './notears')
from notears import utils
#控制随机种子
utils.set_random_seed(0)
#设置矩阵随机的参数d是维度，s0是非0元素个数
n, d, s0, graph_type, sem_type = 100, 3, 1, 'ER', 'gauss'
#得到0，1矩阵
B_true = utils.simulate_dag(d, s0, graph_type)
#随机填充B得到W，默认高斯分布
W_true = utils.simulate_parameter(B_true)
#np.savetxt('W_true2.csv', W_true, delimiter=',')
#根据W生成样本
X = utils.simulate_linear_sem(W_true, n, sem_type)
#np.savetxt('X2.csv', X, delimiter=',')

W_est = solve_all(X,lamda=0.1)
print("-----------------------accuracy---------------------------------")
#检查是否是DAG
assert utils.is_dag(W_est)
#np.savetxt('W_est2.csv', W_est, delimiter=',')
#计算指标
acc = utils.count_accuracy(B_true, W_est != 0)
print(f"accuracy = {acc}")
print("-----------------------compare---------------------------------")
print(f'estimated:\n{W_est}')
print("--------------------------------------------------------")
print(f"true:\n:{W_true}")
print("--------------------------------------------------------")


0.003497235323917327 267.2251134538994
0 0 0.0 277.38077862578706
0 2 0.0 303.6119683838644
1 1 0.0 281.23285080064613
1 2 0.0 267.3934165065138
2 0 0.0 314.7754418236585
2 1 0.0 267.4450869491839
2 2 0.0 608.6408956338087
iter: 0 267.3934165065138 8.107337927389697 chosen: 1 2 0.0
[[ 0.078944    0.0034418   0.15586116]
 [ 0.0034418   0.08371552  0.        ]
 [ 0.15586116 -0.01044511  0.41504964]]
0 0 0.0 277.5490816784014
0 1 0.01 267.34322084360554
0 2 0.0 303.7721546085477
1 1 0.0 281.40115385326044
2 0 0.0 314.9437448762728
2 2 0.0 608.9296301330328
iter: 1 267.34322084360554 8.107337927389693 chosen: 0 1 0.01
[[ 0.078944    0.01        0.15586116]
 [ 0.0034418   0.08371552  0.        ]
 [ 0.15586116 -0.01044511  0.41504964]]
0 0 0.0 277.49888601549304
0 2 0.0 303.7219589456394
1 1 0.0 281.34822086639434
2 0 0.0 314.89354921336457
2 2 0.0 608.8794344701244
iter: 2 277.49888601549304 5.300450554543119 chosen: 0 0 0.0
[[ 0.          0.01        0.15586116]
 [ 0.0034418   0.08371552  